# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [2]:
import copy
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from unityagents import UnityEnvironment
from collections import namedtuple, deque

In [ ]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

# get the default brain
wimbledon = env.brain_names[0]
brain = env.brains[wimbledon]

env_info = env.reset(train_mode=True)[wimbledon]

action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]

In [ ]:
env.close()

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
ep_len = np.zeros((5), dtype='int32')
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[wimbledon]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        ep_len[i-1]+=1
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[wimbledon]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

In [ ]:
state_size

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

## Hyperparameters

In [ ]:
BUFFER_SIZE = int(3e6)     # replay buffer size
BATCH_SIZE = 8          # minibatch size
GAMMA = 0.99               # discount factor
TAU = 1e-3                 # for soft update of target parameters
LR_ACTOR = 1e-4            # learning rate of the actor 
LR_CRITIC = 2e-4           # learning rate of the critic
update_every = 20          # number of timesteps after which to run an update
SN = 0.25                  # starting value for additive noise scale (exploratory actions)
ND = 0.99999               # noise decay rate (exploratory actions)
NM = 0.01                  # noise minimum to be maintained (exploratory actions)

## DDPG agents
____
### Define Actor and Critic

In [ ]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size, hu=(400, 300), activ_in = F.relu, activ_out = torch.tanh):
        super(Actor, self).__init__()
        
        self.activ_in = activ_in
        self.activ_out = activ_out
        
        self.input_layer = nn.Linear(state_size, hu[0])
        self.hl1 = nn.Linear(hu[0], hu[1])
        self.output_layer = nn.Linear(hu[-1], action_size)
        
    def forward(self, state):
        x = state
        x = self.activ_in(self.input_layer(x))
        x = self.activ_in(self.hl1(x))
        return self.activ_out(self.output_layer(x))  
    
class Critic(nn.Module):
    def __init__(self, state_size, action_size, hu=(400, 300), activ_in = F.relu):
        super(Critic, self).__init__()
        
        self.activ_in = activ_in
        
        self.input_layer = nn.Linear(state_size, hu[0])
        self.hl1 = nn.Linear(hu[0]+action_size, hu[1])        
        self.output_layer = nn.Linear(hu[-1], 1)
        
    def forward(self, state, action):
        x = state
        u = action
        
        x = self.activ_in(self.input_layer(x))
        x = torch.cat((x, u), dim=1)
        x = self.activ_in(self.hl1(x))
        return self.output_layer(x)

### Define DDPG agent

In [ ]:
class DDPG_Agent():
    def __init__(self, state_size, action_size, start_noise=SN, noise_decay=ND, noise_min=NM, add_noise=True):
        self.state_size = state_size
        self.action_size = action_size
        
        ### Initialise actor online and target networks
        self.actor_online = Actor(state_size, action_size).to(device)
        self.actor_target = Actor(state_size, action_size).to(device)
        self.actor_optimizer = optim.Adam(self.actor_online.parameters(), lr=LR_ACTOR)
        
        ### Initialise critic online and target networks
        self.critic_online = Critic(state_size, action_size).to(device)
        self.critic_target = Critic(state_size, action_size).to(device)
        self.critic_optimizer = optim.Adam(self.critic_online.parameters(), lr=LR_CRITIC)
        
        
        ### Noise parameters for exploration
        self.noise_scale = start_noise
        self.noise_decay = noise_decay
        self.noise_min = noise_min
        
        self.add_noise = add_noise
        
        ### Replay buffer
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        
        ### Keep track of timesteps since last training update
        self.update = 0
    
    ### Generate random normal noise to add to the action output and enable exploration
    def generate_noise(self):
        noise = np.random.normal(loc=0, scale=self.noise_scale, size=self.action_size)
        self.noise_scale = max(self.noise_decay*self.noise_scale, self.noise_min)
        return noise
    
    ### Store experiences in the replay buffer and control training updates
    def step(self, state, action, reward, next_state, done, update_cycles=3):
        # Commit experience to memory
        self.memory.add(state, action, reward, next_state, done)
        self.update = (self.update +1)%update_every
        
        # Run optimisation 'update_cycles' times
        c=0
        if (self.update==0):
            if len(self.memory) > BATCH_SIZE:
                while c < update_cycles:
                    experiences = self.memory.sample()
                    self.learn(experiences, GAMMA)
                    c+=1
    
    ### Use the actor network to select an action. OPTIONAL: add noise to the action (add_noise=True)
    def act(self, state):
        state = torch.from_numpy(state).float().to(device)
        self.actor_online.eval()
        with torch.no_grad():
            action = self.actor_online(state).cpu().data.numpy()
        self.actor_online.train()
        if self.add_noise:
            action += self.generate_noise()
        return np.clip(action, -1, 1)
    
    ### Run a training update from a batch of experiences on the critic and actor online networks and a 
    ## soft update on the respective target networks
    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences
        
        ### Update critic
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        Q_targets = rewards + (gamma * Q_targets_next * (1-dones))
        Q_expected = self.critic_online(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        # Backprop
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_online.parameters(), 1)
        self.critic_optimizer.step()
        
        
        ### Update actor
        actions_pred = self.actor_online(states)
        actor_loss = -self.critic_online(states, actions_pred).mean()
        
        # Backprop
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.actor_online.parameters(), 1)
        self.actor_optimizer.step()
        
        
        ### Update targets
        self.soft_update(self.critic_online, self.critic_target, TAU)
        self.soft_update(self.actor_online, self.actor_target, TAU)
    
    ### Blend together online and target network parameters
    def soft_update(self, online_model, target_model, tau):
        for target_param, online_param in zip(target_model.parameters(), online_model.parameters()):
            target_param.data.copy_(tau*online_param.data + (1.0-tau)*target_param.data)
            
### Define a Replaybuffer class to store experiences, organise and sample from them
class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    ### Adds an experience tuple to memory
    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    ### Samples k (batch size) experience tuples randomly from memory
    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states, actions, rewards, next_states, dones)
    
    def __len__(self):
        return len(self.memory)

In [ ]:
### Train a DDPG agent, monitor score and save Actor/Critic checkpoints if task solved.
def MADDPG(output, player1, player2, n_episodes=200, max_t=1000):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=False)[wimbledon]           # reset the environment 
        state_p1 = env_info.vector_observations[0]                     # get the current state
        state_p2 = env_info.vector_observations[1] 
        score = np.array([0,0], dtype='float64')                                                   # initialize the score
        for t in range(max_t):
            action_p1 = player1.act(state_p1)                               # select an action
            action_p2 = player2.act(state_p2)
            combined_a = np.concatenate((action_p1, action_p2))
            env_info = env.step(combined_a)[wimbledon]                 # send action to tne environment
            next_state_p1 = env_info.vector_observations[0]            # get next state
            next_state_p2 = env_info.vector_observations[1]
            reward_p1 = env_info.rewards[0]                            # get reward
            reward_p2 = env_info.rewards[1]
            collab_reward = sum(env_info.rewards)
            done = env_info.local_done                           # check if episode finished
            player1.step(state_p1, action_p1, reward_p1, next_state_p1, done[0])     # agent takes one step to train
            player2.step(state_p2, action_p2, reward_p2, next_state_p2, done[1])
            state_p1 = next_state_p1                                      # roll over state to next time step
            state_p2 = next_state_p2
            score += env_info.rewards                                         # update the score
            if done:                                                # exit loop if episode finished
                break 
        scores_deque.append(np.max(score))
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.3f}\tScore: {:.3f}'.format(i_episode, np.mean(scores_deque), np.max(score)), end="")
        #print('Federer {:.4f}-{:.4f} Murray'.format(scores[-1][0], scores[-1][1]))
        if np.mean(scores_deque)>=0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            #torch.save(agent.actor_online.state_dict(), '{}_checkpoint_actor.pth'.format(output))
            #torch.save(agent.critic_online.state_dict(), '{}_checkpoint_critic.pth'.format(output))
            break
            
    return scores

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
federer = DDPG_Agent(state_size, action_size)
murray = DDPG_Agent(state_size, action_size)

scores = MADDPG(test, federer, murray)

In [ ]:
testscores = []
testscore = np.array([5,0])
testdeque = deque(maxlen=100)
testdeque.append(np.max(testscore))
testscores.append(testscore)

In [ ]:
testscores

In [ ]:
testdeque

In [ ]:
testscores

In [ ]:
testscores[-1][0]

In [ ]:
np.max(testscores)